# MTTS-CAN

This notebook demonstrates how to extract respiration and pulse from a video using the MTTS-CAN model.

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband05'
setting = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_bgr(subject, setting, show_progress=True)

## Preprocessing

The MTTS-CAN model expects the frames to be resized and normalized in the temporal domain.

In [ ]:
from respiration.extractor.mtts_can import (
    preprocess_video_frames,
    preprocess_frames_original,
)

# resized, normalized = preprocess_frames_original(frames)
resized, normalized = preprocess_video_frames(frames)

In [ ]:
import matplotlib.pyplot as plt

# Plot resized and normalized frames
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(resized[1])
axs[0].set_title('Resized Frame')

axs[1].imshow(normalized[1])
axs[1].set_title('Normalized Frame')

## Prediction

In [ ]:
import matplotlib.pyplot as plt

from respiration.extractor.mtts_can import load_model

frame_depth = 10

# The model expects a number of frames that is a multiple of frame_depth
cut_off = (normalized.shape[0] // frame_depth) * frame_depth
input_resized = resized[:cut_off]
input_normalized = normalized[:cut_off]

In [ ]:
model = load_model(
    frame_depth=frame_depth,
)
predictions = model.predict(
    (input_resized, input_normalized),
    batch_size=100
)

In [ ]:
import numpy as np

pulse_prediction = np.cumsum(predictions[0])
respiration_prediction = np.cumsum(predictions[1])

## Show the predicted pulse

In [ ]:
import respiration.preprocessing as sp

pulse_pred = sp.detrend_tarvainen(pulse_prediction, 100)
pulse_pred = sp.butterworth_filter(pulse_pred, meta.fps, 0.75, 2.5)

In [ ]:
# Plot the pulse 
plt.plot(pulse_pred)
plt.title('Predicted Pulse')
plt.xlabel('Time')
plt.ylabel('Pulse')
plt.show()

## Show the predicted respiration

In [ ]:
respiration_prediction_processed = sp.standard_processing(respiration_prediction, meta.fps)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20, 10))

# Add some space between the plots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(respiration_prediction, label='Respiration Raw')
axs[0].set_title('Respiration Raw')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Respiration')
axs[0].legend()

axs[1].plot(respiration_prediction_processed, label='Respiration Preprocessed')
axs[1].set_title('Respiration Preprocessed')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Respiration')
axs[1].legend()